# Import packages

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import pandas as pd
import time
from credentials import * # make sure to set the required files
from csv import writer

In [2]:
# Ignore warning messages
import warnings
warnings.filterwarnings("ignore")

# Login and connect

In [3]:
es = Elasticsearch(hosts=hosts, # make sure to configure the hosts in the credentials file
                   verify_certs=False,
                   http_auth=(username, password), # configure the login details in the credentials.py file
                   #api_key=(api_username, api_password), # configure the api login details in the credentials.py file
                  )

In [4]:
# Check API response 
elastic_info = es.info()
for k, v in elastic_info.items():
    print(k, '-'*(20-(1*len(k))), v)

AuthorizationException: AuthorizationException(403, 'security_exception', 'action [cluster:monitor/main] is unauthorized for user [ashek] with roles [read_index,machine_learning_user], this action is granted by the cluster privileges [monitor,manage,all]')

In [ ]:
#Check health of elastic cluster
es_health = es.cluster.health(wait_for_status='yellow', request_timeout=1)
for k, v in es_health.items():
    print(k, '-'*(20-(1*len(k))), v)

# Check the list of Indices and columns

In [4]:
for i in es.indices.get_mapping().keys():
    print(i)

gstt_clinical_documents_letters
gstt_clinical_documents_edl
gstt_clinical_noting_20220105
gstt_clinical_neonatal_comments
gstt_clinical_cancer_documents_20220120
gstt_clinical_documents_letters_annotations
gstt_clinical_epr_orders_20220120
gstt_clinical_epr_observations_20220120
gstt_clinical_epr_results_20220120
gstt_clinical_cancer_documents_annotations
gstt_clinical_epr_results
gstt_clinical_emergency_20211224
gstt_clinical_documents_edl_annotations


In [5]:
# Check the list of columns in that index
index = 'gstt_clinical_documents_letters'
for col in es.indices.get_mapping(index=index)[index]['mappings']['properties'].keys():
    print(col)

body_analysed
document_ApplicationCode
document_ApplicationSource
document_ArrivalDtm
document_Author
document_AuthoredDtm
document_AuthoredProviderGUID
document_AuthorisorDisplayName
document_AuthorisorGUID
document_CareProviderDisplayName
document_CareProviderTypeCode
document_ChartGUID
document_Comment
document_ContentType
document_CreatedBy
document_CreatedWhen
document_DateAdded
document_DateCreated
document_DefinitionGUID
document_Description
document_Entered
document_EnteredDisplayName
document_EnteredOccupationCode
document_EntryType
document_Extension
document_ExternalDocumentID
document_FileName
document_FillerDocumentID
document_FillerFacilityID
document_GUID
document_KTreeRootGUID
document_LastUpdated
document_LetterMakerGUID
document_Name
document_OrganizationalUnit
document_ReviewMasterCategory
document_ReviewSubCategory
document_Source
document_TextType
document_TouchedBy
document_TouchedWhen
document_Type
document_UserGUID
id
meta_document_analysed
nlp
patientAddress_Ci

# Set parameters

In [6]:
#data = pd.read_csv("path to csv")
pt_list = ['8032007P', '4781588Q', '5462414H'] # example list of patients' patient_TrustNumber here

## Columns of interest

In [7]:
columns = ["body_analysed",
           "patient_RaceCode",
           "patient_Id", # this number can be different from the trust number which you are searching for
           "patient_DOB", 
           "patient_GenderCode", 
           "patient_MaritalStatusCode", 
           "patient_ReligionCode", 
           "patient_DeceasedDtm", 
           "patient_LastName",
           "patient_FirstName",
          ]

## Build query

In [8]:
query = {
    "from" : 0 ,
    "size" : 10000,
    "query": {
        "bool": {
            "filter" : {
                "terms" : {"patient_TrustNumber" : pt_list}
            }
        }
    },
    "_source": columns
}

# Search, retrieve, and save

In [80]:
search_results = scan(es,
                      index=["gstt_clinical_epr_*"],
                      preserve_order=True,  # This has major effects on performance but preserves the search order
                      query=query,
                     )

### Simple search

In [ ]:
# Construct a df
temp_results = []

for hit in search_results:
    row = {}
    row['_index'] = hit['_index']
    row['_type'] = hit['_type']
    row['_id'] = hit['_id']
    row['_score'] = hit['_score']
    row.update(hit['_source'])
    temp_results.append(row)

df = pd.DataFrame(temp_results)
df_headers = ['_index', '_type', '_id', '_score']
df_headers.extend(columns)
output_df = pd.DataFrame(columns=df_headers)
df = pd.concat([output_df, df])

### Simple search and save to file
This option is for large searches

In [81]:
# Construct a df
search_results_filename = 'search_results.csv'  # change output filename
temp_results = []
counter = 0
save_interator = 5000  # saves every x hits
df_headers = ['_index', '_type', '_id', '_score']
df_headers.extend(columns)

with open(search_results_filename, 'a') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(df_headers)
    for hit in search_results:
        if counter % 5000 == 0:
            writer_object.writerows(temp_results)
            temp_results = []
        row = {}
        row['_index'] = hit['_index']
        row['_type'] = hit['_type']
        row['_id'] = hit['_id']
        row['_score'] = hit['_score']
        row.update(hit['_source'])
        temp_results.append(row.values())
        counter += 1
        
    writer_object.writerows(temp_results)
    temp_results = []
    f_object.close()

# Process

In [82]:
# Whatever you want here
df = pd.read_csv(search_results_filename)

In [ ]:
df.head()

# Save

In [ ]:
df.to_csv("file_name.csv", encoding="utf-8", index=False)